Converter for the Financial Filings at the Sec

In [6]:
from zipfile import ZipFile

with ZipFile('importFiles/2010q1.zip') as myzip:
    with myzip.open('num.txt') as myfile:
        print(myfile.read())
    with myzip.open('pre.txt') as myfile:
        print(myfile.read())
    with myzip.open('sub.txt') as myfile:
        print(myfile.read())
    with myzip.open('tag.txt') as myfile:
        print(myfile.read())

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

